# Implement VAR to yahoo finance Week 1

In [1]:
import pandas as pd
import numpy as np

## Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import acf, pacf, grangercausalitytests
from statsmodels.tsa.statespace.varmax import VARMAX

## Import TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit

import yfinance as yf

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

## def of prediction_class

In [2]:
def prediction_class(df):
    df['Prediction'] = np.where(df['Close'] < df['Close'].shift(-1), 1, 0)
    return df

## Import from Yahoo Finance

In [3]:
Ford = yf.download('F', period="3mo")

[*********************100%***********************]  1 of 1 completed


In [4]:
Ford['change_in_price'] = Ford['Close'].diff()
Ford

,Open,High,Low,Close,Adj Close,Volume,change_in_price
Date,,,,,,,
2024-08-09,10.08,10.17,9.97,10.10,9.964611,48860800,NaN
2024-08-12,10.10,10.10,9.84,9.86,9.727828,56118700,-0.240001
2024-08-13,9.92,10.15,9.77,10.14,10.004075,44766100,0.280001
2024-08-14,10.16,10.24,10.05,10.18,10.043539,37026400,0.040000
2024-08-15,10.37,10.50,10.37,10.46,10.319785,41597900,0.280000
...,...,...,...,...,...,...,...
2024-11-04,10.23,10.57,10.23,10.36,10.221126,55323800,0.139999
2024-11-05,10.33,10.64,10.30,10.60,10.457909,49309000,0.240001
2024-11-06,10.95,11.23,10.84,11.19,11.039999,100781800,0.589999


In [5]:
Toyota = yf.download('TM', period="3mo")
Toyota['change_in_price'] = Toyota['Close'].diff()

[*********************100%***********************]  1 of 1 completed


## Split the data

In [6]:
df_F_test = Ford[-5:].copy()
df_F_test

,Open,High,Low,Close,Adj Close,Volume,change_in_price
Date,,,,,,,
2024-11-04,10.23,10.57,10.23,10.36,10.221126,55323800,0.139999
2024-11-05,10.33,10.64,10.30,10.60,10.457909,49309000,0.240001
2024-11-06,10.95,11.23,10.84,11.19,11.039999,100781800,0.589999
2024-11-07,11.01,11.11,10.83,10.96,10.960000,55329600,-0.230000
2024-11-08,10.93,10.97,10.86,10.97,10.970000,42822846,0.010000


In [8]:
df = pd.concat([Ford['change_in_price'][: -5], Toyota['change_in_price'][: -5]], axis = 1)

In [9]:
df.columns = ['F_change_in_price', 'TM_change_in_price']

In [10]:
df

,F_change_in_price,TM_change_in_price
Date,,
2024-08-09,NaN,NaN
2024-08-12,-0.240001,-0.619995
2024-08-13,0.280001,4.419998
2024-08-14,0.040000,4.619995
2024-08-15,0.280000,4.779999
2024-08-16,0.050000,0.880005
2024-08-19,0.210000,1.029999
2024-08-20,-0.040000,-2.360001
2024-08-21,0.170000,2.279999


## Apply VAR model

In [12]:
model = VAR(df.dropna())
results = model.fit(22)
predictions = results.forecast(df.values[-22:], steps=5)[:, 0]

C:\Users\teres\anaconda3\envs\equity_vs_commodity\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [13]:
predictions

array([-0.351018  , -0.08111572, -0.54356562,  0.08491384,  0.13430611])

In [14]:
df_F_test

,Open,High,Low,Close,Adj Close,Volume,change_in_price
Date,,,,,,,
2024-11-04,10.23,10.57,10.23,10.36,10.221126,55323800,0.139999
2024-11-05,10.33,10.64,10.30,10.60,10.457909,49309000,0.240001
2024-11-06,10.95,11.23,10.84,11.19,11.039999,100781800,0.589999
2024-11-07,11.01,11.11,10.83,10.96,10.960000,55329600,-0.230000
2024-11-08,10.93,10.97,10.86,10.97,10.970000,42822846,0.010000


In [15]:
accuracy_TM = sum(predictions*df_F_test['change_in_price'] > 0)/5

In [16]:
accuracy_TM

0.2